- title: description of work
- toc: true
- categories: [1.D, 2.B, 3.B, C3.1]
- image: /images/monkeys.png
- type: pbl
- week: 34

In [ ]:
""" database dependencies to support sqliteDB examples """
from random import randrange
from datetime import date
import os, base64
import json

from __init__ import app, db
from sqlalchemy.exc import IntegrityError
from sqlalchemy import Column, String
import base64

''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into Python shell and follow along '''


class Images(db.Model):
    __tablename__ = 'images'  # table name is plural, class name is singular

    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _uid = db.Column(db.String(255), unique=True, nullable=False)
    _likes = db.Column(db.Integer, unique=False, nullable=True)
    _dob = db.Column(db.Date)
    _image = db.Column(db.String(50000), unique=False, nullable=True)

    # Defines a relationship between User record and Notes table, one-to-many (one user to many notes)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, name, uid, likes, dob=date.today(), image=None):
        self._name = name    # variables with self prefix become part of the object, 
        self._uid = uid
        self._likes = likes
        self._dob = dob
        self._image = image

    @property
    def image(self):
        return self._image

    @image.setter
    def image(self, image):
        self._image = image
    # a name getter method, extracts name from object
    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter
    def name(self, name):
        self._name = name
    
    # a getter method, extracts email from object
    @property
    def uid(self):
        return self._uid
    
    # a setter function, allows name to be updated after initial object creation
    @uid.setter
    def uid(self, uid):
        self._uid = uid
        
    # check if uid parameter matches user id in object, return boolean
    def is_uid(self, uid):
        return self._uid == uid
    
    @property
    def likes(self):
        return self._likes
    
    # a setter function, allows name to be updated after initial object creation
    @likes.setter
    def likes(self, likes):
        self._likes = likes
    
    # dob property is returned as string, to avoid unfriendly outcomes
    @property
    def dob(self):
        dob_string = self._dob.strftime('%m-%d-%Y')
        return dob_string
    
    # dob should be have verification for type date
    @dob.setter
    def dob(self, dob):
        self._dob = dob
    
    # output content using str(object) in human readable form, uses getter
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            db.session.add(self)
            db.session.commit()
            return self
        except IntegrityError:
            db.session.remove()
            return None


    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "uid": self.uid,
            "dob": self.dob,
            "likes": self.likes,
            "image": self.image
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, name="", uid="", likes=""):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(uid) > 0:
            self.uid = uid
        if likes > 0:
            self.likes = likes
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None


"""Database Creation and Testing """


# Builds working data for testing
def initImages():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = Images(name='Beach', uid='toby', likes=4, dob=date(1847, 2, 11), 

The provided code is a Python script that defines a database model and performs basic CRUD (Create, Read, Update, Delete) operations on a table called 'images'. It uses the SQLAlchemy library to interact with the SQLite database.

Here's a breakdown of the code:

1. Imports: The necessary libraries and modules are imported, including `randrange` from `random`, `date` from `datetime`, `os`, `base64`, and `json`.

2. Database Model: The `Images` class is defined as a subclass of `db.Model`, which represents a table in the database. The table name is specified as 'images'. The class defines columns (`id`, `_name`, `_uid`, `_likes`, `_dob`, `_image`) using the SQLAlchemy's `Column` class.

3. Constructor: The `__init__` method is the constructor of the `Images` class. It initializes the instance variables of the object (self) with the provided arguments (`name`, `uid`, `likes`, `dob`, `image`).

4. Property Decorators: The class defines property decorators (`@property`) and corresponding setter methods (`@<property_name>.setter`) for the private attributes (`_name`, `_uid`, `_likes`, `_dob`, `_image`). These decorators allow accessing and updating the attributes as properties of the object.

5. Getter Methods: The class defines getter methods (`@property`) for the properties (`name`, `uid`, `likes`, `dob`, `image`), which extract the values from the corresponding private attributes.

6. Utility Methods: The class defines additional utility methods such as `is_uid`, `read`, `create`, `update`, `delete`, and `__str__`, which perform CRUD operations on the table and provide the necessary functionality.

7. Database Creation and Testing: The `initImages` function is defined to initialize the database and add some test data. It creates the database tables (`db.create_all()`) and adds a test record to the `Images` table (`u1 = Images(name='Beach', uid='toby', likes=4, dob=date(1847, 2, 11)`).

Please note that the code snippet provided is incomplete and ends abruptly with an incomplete line of code (`explain`). If you have any specific questions or need further explanation, feel free to ask.

In [ ]:
import json
from flask import Blueprint, request, jsonify
from flask_restful import Api, Resource # used for REST API building
from datetime import datetime

from model.images import Images
from operator import itemgetter

image_api = Blueprint('image_api', __name__,
                   url_prefix='/api/images')

# API docs https://flask-restful.readthedocs.io/en/latest/api.html
api = Api(image_api)

class UserAPI:        
    class _CRUD(Resource):  # User API operation for Create, Read.  THe Update, Delete methods need to be implemeented
        def post(self): # Create method
            ''' Read data for json body '''
            body = request.get_json()
            
            ''' Avoid garbage in, error checking '''
            # validate name
            name = body.get('name')
            if name is None or len(name) < 2:
                return {'message': f'Name is missing, or is less than 2 characters'}, 400
            # validate uid
            uid = body.get('uid')
            if uid is None or len(uid) < 2:
                return {'message': f'User ID is missing, or is less than 2 characters'}, 400
            # look for password and dob
            likes = body.get('likes')
            dob = body.get('dob')
            image = body.get('image')

            ''' #1: Key code block, setup USER OBJECT '''
            uo = Images(name=name, 
                      uid=uid, likes = likes, image = image)
            
            ''' Additional garbage error checking '''
            # set password if provided
            # convert to date type
            if dob is not None:
                try:
                    uo.dob = datetime.strptime(dob, '%Y-%m-%d').date()
                except:
                    return {'message': f'Date of birth format error {dob}, must be mm-dd-yyyy'}, 400
            
            ''' #2: Key Code block to add user to database '''
            # create user in database
            img = uo.create()
            # success returns json of user
            if img:
                return jsonify(img)
            # failure returns error
            return {'message': f'Processed {name}, either a format error or User ID {uid} is duplicate'}, 400

        def get(self):
            images = Images.query.all()
            json_ready = [image.read() for image in images]

            # Sort the images based on the number of likes using merge sort
            sorted_images = self.merge_sort(json_ready, key=itemgetter('likes'))

            return jsonify(sorted_images)

        def merge_sort(self, arr, key):
            if len(arr) <= 1:
                return arr

            mid = len(arr) // 2
            left_half = arr[:mid]
            right_half = arr[mid:]

            left_half = self.merge_sort(left_half, key=key)
            right_half = self.merge_sort(right_half, key=key)

            return self.merge(left_half, right_half, key=key)

        def merge(self, left, right, key):
            merged = []
            left_index = 0
            right_index = 0

            while left_index < len(left) and right_index < len(right):
                if key(left[left_index]) < key(right[right_index]):
                    merged.append(left[left_index])
                    left_index += 1
                else:
                    merged.append(right[right_index])
                    right_index += 1

            merged.extend(left[left_index:])
            merged.extend(right[right_index:])

            return merged
        
        def patch(self):
            body = request.get_json()
            id = body.get('id')
            image = Images.query.filter_by(id=id).first()
            try:
                name = body.get('name')
                uid = body.get('uid')
                likes = body.get('likes')
                image.update(uid=uid, likes=likes, name=name)
                return jsonify(image.get())
            except:
                print(f"error with {id}")



    
 

    # building RESTapi endpoint
    api.add_resource(_CRUD, '/')

The provided code defines a REST API using the Flask framework and Flask-RESTful extension. The API allows performing CRUD operations on the `Images` table.

Here's a breakdown of the code:

1. Imports: The necessary libraries and modules are imported, including `json`, `Blueprint`, `request`, `jsonify` from Flask, `Api` and `Resource` from Flask-RESTful, and `datetime`.

2. Blueprint and API Setup: The `image_api` blueprint is created using `Blueprint` with the URL prefix '/api/images'. An instance of `Api` is created with the `image_api` blueprint.

3. API Resource: The `_CRUD` class is defined as a subclass of `Resource`. It represents the API operations for creating and reading images from the database. The `post` method handles the creation of a new image, and the `get` method retrieves all images from the database.

4. `post` Method: This method is used to create a new image record in the database. It reads the JSON data from the request body and performs validation on the required fields (`name` and `uid`). If the validation passes, a new `Images` object (`uo`) is created and added to the database using the `create` method. The created image is returned as JSON if successful, or an error message if there's a failure.

5. `get` Method: This method retrieves all images from the database. It queries the `Images` table using `Images.query.all()` and converts each image object to a dictionary using the `read` method. The images are then sorted based on the number of likes using the `merge_sort` method, and the sorted images are returned as JSON.

6. Sorting Method: The `_CRUD` class defines the `merge_sort`, `merge`, and `patch` methods. The `merge_sort` method implements the merge sort algorithm to sort the images based on the provided key (in this case, the number of likes). It recursively splits the list into halves, sorts each half, and merges them back together in sorted order. The `merge` method merges two sorted halves. These methods are used in the `get` method to sort the images based on likes.

7. `patch` Method: This method handles the partial update (PATCH) operation for an image. It reads the JSON data from the request body, extracts the image ID, and retrieves the corresponding image from the database using `Images.query.filter_by(id=id).first()`. Then, it updates the image's attributes (`name`, `uid`, `likes`) using the `update` method. If successful, the updated image is returned as JSON.

8. API Endpoint: The `_CRUD` resource is added to the API with the URL endpoint '/'.

Please note that the code snippet ends abruptly with an incomplete line of code (`explain`). If you have any specific questions or need further explanation, feel free to ask.